In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from nltk.stem import WordNetLemmatizer
from timeit import default_timer as timer


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Mohamed
[nltk_data]     Mousa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mohamed
[nltk_data]     Mousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def Load_data(document_path):
    
    with open(document_path, 'r') as file:
        document = file.read()
    return document

In [3]:
Data = []
import os
for dirname, _, filenames in os.walk('D:\cysheild\Assignment\Assignment_NLP\Assignment_NLP\Semantic_keywords\politics'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        doc_class = filename.split('_')[0].lower()
        doc_titles = filename
        documents = Load_data(os.path.join(dirname, filename))
        
        Data.append( documents )

In [4]:
len(Data)

100

In [5]:
stemmer = WordNetLemmatizer()

def preprocess(x):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(x))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # word tokenize
    document = word_tokenize(document)
    # remove stop words
    document = [word for word in document if word not in stopwords.words('english')]
    # word lemmatization
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

In this code, we define a function called preprocess that takes a text as input, preprocesses it using nltk, filters out stop words and punctuation, and returns a string of the remaining words.

We then define a function called search_words_tfidf that takes a text and a list of words to search for as input. We preprocess the input text and words using preprocess function, and then vectorize them using Tf-Idf.

We compute the cosine similarity between the vectorized text and the vectorized words using cosine_similarity function. We extract the words with highest similarity greater than 0 and return them as the matched words.

Finally, we display the list of matched words. Note that the threshold for cosine similarity can be adjusted depending on the desired level of matching

In [6]:
def search_words_tfidf(Data, words):
    start = timer()
    # preprocess input text and words
    text = preprocess(Data)
    words = [preprocess(word) for word in words]
    # vectorize text and words using Tf-Idf
    vectorizer = TfidfVectorizer().fit([text] + words)
    text_vector = vectorizer.transform([text])
    word_vectors = vectorizer.transform(words)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(Data)
    df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names())
    q_vec = vectorizer.transform(words).toarray().reshape(df.shape[0],)

    
    # compute cosine similarity between text and words
    similarities = cosine_similarity(text_vector, word_vectors)
    # extract matched words with highest similarity
    matched_words = [words[i] for i, similarity in enumerate(similarities[0]) if similarity > 0 ]
    
    med = timer()
    
    print(matched_words)
    print("Time Taken for searching for the words :  " + str(med - start))  
    
    
    sim = {}
    for i in range(10):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)    
    
    for k, v in sim_sorted:
        if v != 0.0:
            print("Similarity Value:", v)
            print(Data[k])
            print()
    
    print("Time Taken for searching for the article :  " + str(timer() - start))  

In [7]:

words = ['The government is bad' ]



search_words_tfidf(Data, words)



['government bad']
Time Taken for searching for the words :  10.916880799999998
Similarity Value: 0.02658117660357745
Council tax rise 'reasonable'

Welsh councils should set their taxes at "reasonable levels" after being given an average funding increase of 6%, says the assembly government.

Finance Minister Sue Essex said it was a "good deal" for local government. The Â£3.2bn settlement includes the "full" Â£7.4m from the UK Treasury announced by Chancellor Gordon Brown. But opposition parties said rebanding of council taxes would mean steep rises. In addition, Â£13.4m will come from the business support grant - a scheme which enables local authorities to keep part of business rates. She said where spending rises were kept to around 5% she was "confident that councils will be able to set council taxes at reasonable levels." The Welsh Local Government Association (WLGA) had said on the eve of the announcement said that "significant" cuts to services may still be unavoidable. After the

C:\Users\Mohamed Mousa\anaconda3\myanaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
